In [1]:
import pyarc

In [2]:
import fim

In [3]:
import pandas as pd

In [4]:
import numpy as np

In [5]:
tracks=pd.read_csv("C:/Users/Vincent/Downloads/COMP4710_project/COMP4710_project/my_dataset.csv",encoding='latin1')

In [6]:
tracks

,track_id,title,year,genre,word,count
0,TRAADNA128F9331246,Crossfire,1992,blues,i,4
1,TRAADNA128F9331246,Crossfire,1992,blues,the,14
2,TRAADNA128F9331246,Crossfire,1992,blues,you,1
3,TRAADNA128F9331246,Crossfire,1992,blues,to,3
4,TRAADNA128F9331246,Crossfire,1992,blues,and,1
...,...,...,...,...,...,...
886377,TRZZXFY128F9342D0E,Toca Me,1999,electronic,noon,1
886378,TRZZXFY128F9342D0E,Toca Me,1999,electronic,deeper,1
886379,TRZZXFY128F9342D0E,Toca Me,1999,electronic,miracl,14
886380,TRZZXFY128F9342D0E,Toca Me,1999,electronic,rescu,1


In [7]:
from pyarc import CBA,TransactionDB

In [8]:
words=['i','you','they','we','he','she','it','a','and','the','to','me','not','in','of','on','about','your','that','do','am','for','no','here','there','oh','my','is','are']
indices=np.empty([1],dtype='int64')
for word in words:
    temp=np.concatenate((indices,tracks[tracks['word']==word].index.values))
    indices=temp
tracks2=tracks.drop(index=indices[1:])

In [9]:
singletons=tracks2.groupby('word').sum('count')
bbb=singletons[singletons['count']==1].index.get_level_values(0)
indices=np.empty([1],dtype='int64')
for word in bbb:
    temp=np.concatenate((indices,tracks[tracks['word']==word].index.values))
    indices=temp
tracks3=tracks2.drop(indices[1:])

In [10]:
all_tracks = [track[1]['word'].tolist() for track in list(tracks3.groupby(['track_id','title']))]
lyrics = [track[1]['genre'].tolist()[0] for track in list(tracks3.groupby(['track_id','title']))]
lyrics = pd.DataFrame(lyrics, columns = ["genre"])
lyrics["words"] = all_tracks

In [11]:
lyrics

,genre,words
0,blues,"[be, just, time, come, one, see, got, never, c..."
1,rock,"[will, all, have, love, so, know, with, when, ..."
2,pop,"[will, all, so, this, with, just, when, like, ..."
3,rock,"[all, be, have, love, so, know, just, when, li..."
4,rock,"[will, all, be, have, love, with, like, up, on..."
...,...,...
9995,electronic,"[be, love, know, what, just, come, one, get, i..."
9996,rock,"[will, all, be, have, know, this, but, what, o..."
9997,metal,"[all, be, this, with, what, now, come, was, se..."
9998,rock,"[will, all, have, so, but, with, what, when, n..."


In [12]:
lyrics2=lyrics

In [13]:
lyrics2

,genre,words
0,blues,"[be, just, time, come, one, see, got, never, c..."
1,rock,"[will, all, have, love, so, know, with, when, ..."
2,pop,"[will, all, so, this, with, just, when, like, ..."
3,rock,"[all, be, have, love, so, know, just, when, li..."
4,rock,"[will, all, be, have, love, with, like, up, on..."
...,...,...
9995,electronic,"[be, love, know, what, just, come, one, get, i..."
9996,rock,"[will, all, be, have, know, this, but, what, o..."
9997,metal,"[all, be, this, with, what, now, come, was, se..."
9998,rock,"[will, all, have, so, but, with, what, when, n..."


In [14]:
from mlxtend.preprocessing import TransactionEncoder
te1=TransactionEncoder()
te_ary = te1.fit(lyrics2['words']).transform(lyrics2['words'])


In [15]:
te_ary

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [16]:
df=pd.DataFrame(te_ary)

In [17]:
X=df
y=lyrics2['genre']

In [18]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.8)

In [19]:
from sklearn.tree import DecisionTreeClassifier
tree=DecisionTreeClassifier()
tree.fit(X_train,y_train)
y_pred=tree.predict(X_test)

In [20]:
y_pred

array(['rock', 'rock', 'country', ..., 'rock', 'metal', 'pop'],
      dtype=object)

In [21]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test,y_pred)

0.1638542327791703

In [22]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred=rf.predict(X_test)
balanced_accuracy_score(y_test,y_pred)

0.14315031055538263

In [23]:
from sklearn.svm import SVC
svc=SVC()
svc.fit(X_train,y_train)
y_pred=svc.predict(X_test)
balanced_accuracy_score(y_test,y_pred)

0.16335292580482008

In [24]:
from sklearn.neighbors import KNeighborsClassifier
knc=KNeighborsClassifier()
knc.fit(X_train,y_train)
y_pred=knc.predict(X_test)
balanced_accuracy_score(y_test,y_pred)

0.10316781314011188

In [25]:
from sklearn.linear_model import SGDClassifier
sgd=SGDClassifier()
sgd.fit(X_train,y_train)
y_pred=sgd.predict(X_test)
balanced_accuracy_score(y_test,y_pred)

0.22067870997613767

In [26]:
from sklearn.naive_bayes import GaussianNB
gnb=GaussianNB()
gnb.fit(X_train,y_train)
y_pred=gnb.predict(X_test)
balanced_accuracy_score(y_test,y_pred)

0.17495747993809355